## Library Imports

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing our List of Stocks

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

## Acquiring API Token

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## First API Call
-Market Cap and Stock Price

In [6]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'EAGETSG LSMA N SLROEN/DLKTB)AAC(Q',
 'calculationPrice': 'close',
 'open': 0,
 'openTime': None,
 'openSource': 'faifcilo',
 'close': 0,
 'closeTime': None,
 'closeSource': 'iifcloaf',
 'high': 0,
 'highTime': 1676533849386,
 'highSource': 'c ai 5emerudi1ynt depel',
 'low': 0,
 'lowTime': 1626365605210,
 'lowSource': ' m ru eaeeddpice1tiyn5l',
 'latestPrice': 124.98,
 'latestSource': 'Close',
 'latestTime': 'May 28, 2021',
 'latestUpdate': 1625986189784,
 'latestVolume': None,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexLastUpdated': None,
 'delayedPrice': None,
 'delayedPriceTime': None,
 'oddLotDelayedPrice': None,
 'oddLotDelayedPriceTime': None,
 'extendedPrice': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPriceTime': None,
 'previousClose': 128.74,
 'previousVolume': 99184863,
 'change': -0.68,
 'changePercent': -0.00551,
 'volume': None,
 'iexMarketPercent': None,
 'iex

## Parsing API

In [9]:
data['latestPrice']
data['marketCap']

2095621589850

## Adding Stock Data to Pandas

In [10]:
my_columns = ['Ticker','Stock Price', 'Market Capitalization', 'Number Of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number Of Shares to Buy


In [12]:
final_dataframe = final_dataframe.append(
                                        pd.Series(['AAPL',
                                                  data['latestPrice'],
                                                        data['marketCap'],
                                                        'N/A'],
                                                 index = my_columns),
                                        ignore_index = True)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number Of Shares to Buy
0,AAPL,124.98,2095621589850,N/A


## Looping through Tickers

In [16]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
                                        pd.Series([symbol,
                                                  data['latestPrice'],
                                                        data['marketCap'],
                                                        'N/A'],
                                                 index = my_columns),
                                        ignore_index = True)

In [17]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number Of Shares to Buy
0,A,138.64,42881934815,N/A
1,AAL,25.29,15622455732,N/A
2,AAP,199.10,12540933526,N/A
3,AAPL,130.09,2084332942232,N/A
4,ABBV,118.10,208602931141,N/A
...,...,...,...,...
500,YUM,122.13,36745525652,N/A
501,ZBH,175.45,36669303339,N/A
502,ZBRA,509.32,26881612676,N/A
503,ZION,59.65,9751353721,N/A


## Batch API Calls to Improve Performance

In [18]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [19]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number Of Shares to Buy
0,A,143.41,42324297053,N/A
1,AAL,24.30,15585289506,N/A
2,AAP,193.92,12992205022,N/A
3,AAPL,128.69,2106672730565,N/A
4,ABBV,117.20,202778847744,N/A
...,...,...,...,...
500,YUM,123.97,36322931407,N/A
501,ZBH,172.80,36389289603,N/A
502,ZBRA,505.62,27563115065,N/A
503,ZION,59.13,9497263360,N/A


## Calculating Number of Shares to Buy

In [28]:
portfolio_size = input("Enter the size of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number. \n Try again:")
    portfolio_size = input("Enter the size of your portfolio:")
    

Enter the size of your portfolio:100000


In [29]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])-1):
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe['Stock Price'][i])
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number Of Shares to Buy
0,A,143.41,42324297053,1
1,AAL,24.30,15585289506,8
2,AAP,193.92,12992205022,1
3,AAPL,128.69,2106672730565,1
4,ABBV,117.20,202778847744,1
...,...,...,...,...
500,YUM,123.97,36322931407,1
501,ZBH,172.80,36389289603,1
502,ZBRA,505.62,27563115065,0
503,ZION,59.13,9497263360,3


## Formatting Excel Output

In [30]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

#### Format for File

In [31]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [32]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [33]:
writer.save()